# Этап L5

In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=bec5f5857bb720da8c47d2d888ad3d78cfb0c1ff7a4052a53fd1bca266543e2d
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [5]:
import gc
import pandas as pd
import numpy as np
from numpy import load
from tqdm import tqdm
from random import sample

from lightfm.data import Dataset
from lightfm import LightFM
import lightgbm as lgbm

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import ndcg_score

In [4]:
path = "/content/drive/MyDrive/WB School/data.csv.gzip"
interactions = pd.read_csv(path, compression="gzip")
interactions["order_ts"] = pd.to_datetime(interactions["order_ts"])

После оценки каждой модели следует перезагружать ядро.

# Вспомогательные функции

### Функции для подготовки данных.

Некоторые функции являются общими для всех моделей, поэтому введём их сразу. Остальные будем инициализировать по мере надобности.

Функция `train_test()` делит выборку на обучающую и тестовую в зависимости от модели, которую будем использовать. Для бустинга обучающую выборку надо будет поделить на train/test части ещё раз. **Глобальный test отсекается по времени**.

In [ ]:
def train_test(interactions, for_boosting=False, q=0.7, threshold="2023-02-28 23:59:59.947831"):
  interactions = interactions.drop_duplicates()

  # Обучающая выборка - 2 месяца, тестовая - 1 последний месяц.
  train = interactions[interactions.order_ts <= threshold]
  test = interactions[interactions.order_ts > threshold]

  # Оставляем только "тёплых" пользователей и те товары, взаимодействия с которыми были в train периоде.
  users = np.intersect1d(train.user_id.unique(), test.user_id.unique())
  train = train[train["user_id"].isin(users)]
  test = test[test["user_id"].isin(users)]

  # Для гибридной модели глобальную обучающую выборку разделим на 2 части в соотношении 70:30.
  if for_boosting == True:
    lfm_threshold = train["order_ts"].quantile(q=q, interpolation="nearest")

    lfm_train = train[(train["order_ts"] <= lfm_threshold)]
    lfm_pred = train[(train["order_ts"] > lfm_threshold)]

    users = np.intersect1d(lfm_train.user_id.unique(), lfm_pred.user_id.unique())

    lfm_train = lfm_train[lfm_train["user_id"].isin(users)]
    lfm_pred = lfm_pred[lfm_pred["user_id"].isin(users)]
    test = test[test["user_id"].isin(users)]

    lfm_train = lfm_train.groupby(["user_id", "item_id"], as_index=False).count() \
        .rename(columns={"order_ts": "amount"})

    lfm_pred = lfm_pred.groupby(["user_id", "item_id"], as_index=False).count() \
        .rename(columns={"order_ts": "amount"})

    test = test.groupby(["user_id", "item_id"], as_index=False).count() \
        .rename(columns={"order_ts": "amount"})

    return lfm_train, lfm_pred, test

  elif for_boosting == False:
    train = train.groupby(["user_id", "item_id"], as_index=False).count() \
        .rename(columns={"order_ts": "amount"})

    test = test.groupby(["user_id", "item_id"], as_index=False).count() \
        .rename(columns={"order_ts": "amount"})

    return train, test

Функция `train_test_leave_one_out()` помещает в test последний заказ пользователей и нужна для оценки качества моделей по принципу Leave-One-Out. Параметр path указывает путь к файлу, в котором хранятся user_id пользователей, которые вошли в тестовую выборку при оценке качества на отложенном месяце.

In [ ]:
def train_test_leave_one_out(interactions, path="/content/drive/MyDrive/WB School/L4/users.npy", q=0.7):

  interactions_temporary = interactions.groupby("user_id", as_index=False)["order_ts"].max().rename(columns={"order_ts": "last_order_ts"})
  interactions = pd.merge(
      interactions,
      interactions_temporary,
      on="user_id",
      how="left"
  )

  test = interactions[interactions["order_ts"] == interactions["last_order_ts"]].drop_duplicates(subset=["user_id"]).drop(columns=["last_order_ts"])
  train = interactions[~interactions.index.isin(test.index)].drop(columns=["last_order_ts"])

  lfm_threshold = train["order_ts"].quantile(q=q, interpolation="nearest")

  lfm_train = train[(train["order_ts"] <= lfm_threshold)]
  lfm_pred = train[(train["order_ts"] > lfm_threshold)]

  users = load(path, allow_pickle=True)

  lfm_train = lfm_train[lfm_train["user_id"].isin(users)]
  lfm_pred = lfm_pred[lfm_pred["user_id"].isin(users)]
  test = test[test["user_id"].isin(users)]

  users = np.intersect1d(lfm_train.user_id.unique(), lfm_pred.user_id.unique())
  users = np.intersect1d(users, test.user_id.unique())

  lfm_train = lfm_train[lfm_train["user_id"].isin(users)]
  lfm_pred = lfm_pred[lfm_pred["user_id"].isin(users)]
  test = test[test["user_id"].isin(users)]

  lfm_train = lfm_train.groupby(["user_id", "item_id"], as_index=False).count() \
      .rename(columns={"order_ts": "amount"})

  lfm_pred = lfm_pred.groupby(["user_id", "item_id"], as_index=False).count() \
      .rename(columns={"order_ts": "amount"})

  test = test.groupby(["user_id", "item_id"], as_index=False).count() \
      .rename(columns={"order_ts": "amount"})

  return lfm_train, lfm_pred, test

Функция `lists_to_arrays()` преобразует данные к нужному для вычисления метрик виду.

In [6]:
def lists_to_arrays(ranks, relevances, recommended_items_ids, relevant_items_ids):
  ranks_array = list()
  relevances_array = list()
  recommended_items_ids_array = list()
  relevant_items_ids_array = list()

  for i in tqdm(range(len(ranks))):
    rank = np.array(ranks[i]).astype(float)
    ranks_array.append(rank)

    rel = np.array(relevances[i]).astype(int)
    relevances_array.append(rel)

    rec_items = np.array(recommended_items_ids[i]).astype(int)
    recommended_items_ids_array.append(rec_items)

    rel_items = np.array(relevant_items_ids[i]).astype(int)
    relevant_items_ids_array.append(rel_items)

  ranks_array = np.array(ranks_array)
  relevances_array = np.array(relevances_array)
  recommended_items_ids_array = np.array(recommended_items_ids_array)
  relevant_items_ids_array = np.array(relevant_items_ids_array)

  return ranks_array, relevances_array, recommended_items_ids_array, relevant_items_ids_array

Функция `extract_vectors()` достаёт векторы рангов, релевантностей, id рекомендованных товаров и id релевантных товаров из результирующего на предыдущем шаге датафрейма, а затем преобразует к нужному для вычисления метрик формату.

In [7]:
def extract_vectors(predictions, test, rank_column_name="rank"):

  ranks = predictions.groupby("user_id")[rank_column_name].apply(list).values
  relevances = predictions.groupby("user_id")["relevance"].apply(list).values
  recommended_items_ids = predictions.groupby("user_id")["item_id"].apply(list).values
  relevant_items_ids = test.groupby("user_id")["item_id"].apply(list).values

  ranks, relevances, recommended_items_ids, relevant_items_ids = lists_to_arrays(
      ranks,
      relevances,
      recommended_items_ids,
      relevant_items_ids
  )
  return ranks, relevances, recommended_items_ids, relevant_items_ids

Функция `drop_outlier_items()` исключает из выборки самые популярные товары.

In [19]:
def drop_outlier_items(df, k=20):
  items_df = df.drop(columns="order_ts").groupby("item_id", as_index=False) \
        .count().rename(columns={"user_id": "amount"}).sort_values("amount", ascending=False)
  items = list(items_df.item_id.values[:k])
  df = df[~df["item_id"].isin(items)]

  return df

## Функции для вычисления метрик

### MAP@k

Функция `user_precision()` вычисляет значение AveragePrecision@k для одного пользователя и нужна для вычисления MeanAveragePrecision@k.

In [9]:
def user_average_precision(user_relevances, k=20):
  if user_relevances[:k].sum() == 0:
    return 0
  else:
    # Считаем значение Average Precision@k для одного пользовтаеля.
    average_precision_list = list()

    for k_items in range(1, (k + 1)):
      precision = user_relevances[:k_items].sum()
      precision /= len(user_relevances[:k_items])
      average_precision_list.append(precision)

    average_precision_array = np.array(average_precision_list)
    average_precision = average_precision_array * user_relevances[:k]
    average_precision = average_precision.sum()
    average_precision /= user_relevances[:k].sum()

    return average_precision

Функция `get_map()` считает MeanAveragePrecision@k.

In [10]:
def get_map(relevances, k=20):
  mean_average_precision = 0
  n = len(relevances)

  for i in tqdm(range(n)):
    average_precision = user_average_precision(relevances[i], k)
    mean_average_precision += average_precision

  return mean_average_precision / n

### MAR@k

Функция `eval_single()` считает значение Recal@k для одного пользователя.

In [11]:
def eval_single(recommended_items_ids, relevant_items_ids, k=20):
  recall_at_k = sum(
      [
          1
          for rec_item in recommended_items_ids[:k]
          if rec_item in relevant_items_ids
      ]
  ) / min(len(relevant_items_ids), k)

  return recall_at_k

Функция `user_average_recall()` считает AverageRecall@k одного пользователя.

In [12]:
def user_average_recall(user_relevances_recommended, recommended_items_ids, relevant_items_ids, k=20):
  if user_relevances_recommended[:k].sum() == 0:
    return 0
  else:
    average_recall_list = list()

    for k_items in range(1, (k + 1)):
      recall_at_k = eval_single(recommended_items_ids, relevant_items_ids, k_items)
      average_recall_list.append(recall_at_k)

    average_recall_array = np.array(average_recall_list)
    average_recall_array = average_recall_array * user_relevances_recommended[:k]
    average_recall = average_recall_array.sum()
    average_recall /= min(len(relevant_items_ids), k)

    return average_recall

Функция `get_mar()` считает MeanAverageRecall@k.

In [13]:
def get_mar(relevances_recommended, recommended_items_ids, relevant_items_ids, k=20):
  mean_average_recall = 0
  n = len(relevances_recommended)

  for i in tqdm(range(n)):
    average_recall = user_average_recall(relevances_recommended[i], recommended_items_ids[i], relevant_items_ids[i], k)
    mean_average_recall += average_recall

  return mean_average_recall / n

### NDCG@k

In [14]:
def get_ndcg(ranks_array, relevances_array, is_higher_better=False, k=20):
  n = len(ranks_array)
  ndcg = 0

  # Если на вход подаются скоры, а не ранги, то необходимо указать параметр is_higher_better = True.
  if is_higher_better == False:
    for i in tqdm(range(n)):
      rank = k - ranks_array[i]
      ndcg += ndcg_score([relevances_array[i]], [rank], k=k)
    return ndcg / n

  elif is_higher_better == True:
    for i in tqdm(range(n)):
      ndcg += ndcg_score([relevances_array[i]], [ranks_array[i]], k=k)
    return ndcg / n

### HitRate@k

In [15]:
def get_hit_rate(relevances_array, k=20):
    hit_rate = 0
    n = len(relevances_array)
    for i in tqdm(range(n)):
        hit_rate += relevances_array[i][:k].sum() / k
    hit_rate /= n

    return hit_rate

### 4 в 1

Функция `get_metrics()` считает MAP@k, MAR@k и NDCG@k модели. Выдаёт словарь, где ключи - названия метрик, значения - метрики.

In [16]:
def get_metrics(ranks, relevances, recommended_items_ids, relevant_items_ids, is_higher_better=False, k=20):
  metrics = dict()

  key = "MAP@" + str(k)
  metrics[key] = get_map(relevances, k)

  key = "MAR@" + str(k)
  metrics[key] = get_mar(relevances, recommended_items_ids, relevant_items_ids, k)

  key = "NDCG@" + str(k)
  metrics[key] = get_ndcg(ranks, relevances, is_higher_better, k)

  key = "HitRate@" + str(k)
  metrics[key] = get_hit_rate(relevances, k)

  return metrics

# Popularity Based Recommender

Для модели популярных товаров разбить на train/test, надо иначе, чем для других моделей.

In [23]:
def train_test_popular(interactions, threshold="2023-02-28 23:59:59.947831", path="/content/drive/MyDrive/WB School/L4/users.npy"):
  train = interactions[interactions.order_ts <= threshold]
  test = interactions[interactions.order_ts > threshold]

  users = load(path, allow_pickle=True)

  train = train[train["user_id"].isin(users)]
  test = test[test["user_id"].isin(users)]

  return train, test

In [24]:
train, test = train_test_popular(interactions)

Для Leave-One-Out.

In [37]:
def train_test_popular_loo(interaction, path="/content/drive/MyDrive/WB School/L4/users.npy"):
  interactions_temporary = interaction.groupby("user_id", as_index=False)["order_ts"].max().rename(columns={"order_ts": "last_order_ts"})
  interactions_full = pd.merge(
      interactions,
      interactions_temporary,
      on="user_id",
      how="left"
  )

  test = interactions_full[interactions_full["order_ts"] == interactions_full["last_order_ts"]].drop_duplicates(subset=["user_id"]).drop(columns=["last_order_ts"])
  train = interactions_full[~interactions_full.index.isin(test.index)].drop(columns=["last_order_ts"])

  users = load(path, allow_pickle=True)

  train = train[train["user_id"].isin(users)]
  test = test[test["user_id"].isin(users)]

  return train, test

In [38]:
train, test = train_test_popular_loo(interactions)

Функция `predict_popular()` рекомендует товары на основе их популярности. Сначала всем пользователям рекомендуются преобретёнными в train периоде товары. Затем, если пользователь в течение train периода заказал менее k товаров, список дополняется соотствующим количеством товаров, наиболее популярных среди пользователей train периода.

In [41]:
def predict_popular(train, test, k=20):

  # Приготовим k наиболее популярных товаров среди всех пользователей train периода.
  train_items = train.drop(columns="order_ts").groupby("item_id", as_index=False) \
      .count().rename(columns={"user_id": "amount"}).sort_values("amount", ascending=False)
  items = list(train_items.item_id.values[:k])

  # Заполняем списки рекомендаций пользователей наиболее популярными
  # товарами среди всех пользователей train периода.
  predictions_dict = dict()
  users = test["user_id"].unique()
  for user in tqdm(users):
    predictions_dict[user] = items

  # На основе построенного словаря с рекомедациями составляем датафрейм.
  predictions_df = pd.DataFrame({"user_id": users})
  predictions_df["item_id"] = predictions_dict.values()
  predictions_df = predictions_df.explode("item_id")
  predictions_df["rank"] = predictions_df.groupby("user_id").cumcount() + 1

  test = test.groupby(["user_id", "item_id"], as_index=False).count() \
      .rename(columns={"order_ts": "amount"})
  test = test.drop(columns="amount")
  test["relevance"] = 1

  predictions_df = pd.merge(predictions_df, test, on=["user_id", "item_id"], how="left")
  predictions_df["relevance"] = predictions_df["relevance"].fillna(0)
  predictions_df["relevance"] = predictions_df["relevance"].astype(int)

  return predictions_df, test

In [ ]:
predictions, test = predict_popular(train, test, k=20)

100%|██████████| 514071/514071 [00:00<00:00, 1122301.15it/s]


In [ ]:
ranks_array, relevances_array, recommended_items_ids_array, relevant_items_ids_array = extract_vectors(predictions, test)

100%|██████████| 514071/514071 [00:07<00:00, 64977.92it/s]


На отложенном месяце:

In [ ]:
get_metrics(
    ranks_array,
    relevances_array,
    recommended_items_ids_array,
    relevant_items_ids_array,
    is_higher_better=False,
    k=20
)

100%|██████████| 514071/514071 [00:02<00:00, 240339.42it/s]


{'MAP@20': 0.11347279871752354,
 'MAR@20': 0.047801159744033185,
 'NDCG@20': 0.20017988942199585,
 'HitRate@20': 0.03948854753544846}

Leave-One-Out:

In [ ]:
get_metrics(
    ranks_array,
    relevances_array,
    recommended_items_ids_array,
    relevant_items_ids_array,
    is_higher_better=False,
    k=20
)

100%|██████████| 514071/514071 [00:02<00:00, 240339.42it/s]


{'MAP@20': 0.023951820384126632,
 'MAR@20': 0.11467679217154712,
 'NDCG@20': 0.04335099350809282,
 'HitRate@20': 0.005733839608581156}

# LFM + LGBM

### Несколько вспомогательных функций

Функция `transform_interactions()` понадобится для преобразования данных.

In [ ]:
def transform_interactions(interactions_grouped):
  return interactions_grouped[["user_id", "item_id", "amount"]].itertuples(index=False)

Функция `make_dataset()` преобразует исходные данные к LightFM.Dataset формату и создаёт отображение для оценки качества LightFM моделей.

In [ ]:
def make_dataset(train, test):
  # Создаём обучающий и тестовый датасеты.
  user_ids_buffered = (x for x in train["user_id"].unique())
  item_ids_buffered = (x for x in train["item_id"].unique())

  dataset = Dataset()

  dataset.fit(
    users=user_ids_buffered,
    items=item_ids_buffered
  )

  interaction_matrix_train, _ = dataset.build_interactions(
      transform_interactions(train))

  # Сохраняем отображение.
  lightfm_mapping = dataset.mapping()
  lightfm_mapping = {"users_mapping": lightfm_mapping[0],
                     "items_mapping": lightfm_mapping[2]}

  lightfm_mapping["users_inv_mapping"] = {v: k for k, v in lightfm_mapping["users_mapping"].items()}
  lightfm_mapping["items_inv_mapping"] = {v: k for k, v in lightfm_mapping["items_mapping"].items()}

  all_cols = list(lightfm_mapping["items_mapping"].values())

  return interaction_matrix_train, lightfm_mapping, all_cols

Функция `generate_lightfm_recs_mapper()` является вспомогательной для функции отбора кандидатов от модели, следующей за ней.

In [ ]:
def generate_lightfm_recs_mapper(model, item_ids, known_items, user_features, item_features, N, user_mapping, item_inv_mapping, num_threads=4):
  def _recs_mapper(user):
    user_id = user_mapping[user]
    recs = model.predict(user_id, item_ids, user_features=user_features, item_features=item_features, num_threads=num_threads)

    additional_N = len(known_items[user_id]) if user_id in known_items else 0
    total_N = N + additional_N
    top_cols = np.argpartition(recs, -np.arange(total_N))[-total_N:][::-1]

    final_recs = [item_inv_mapping[item] for item in top_cols]
    if additional_N > 0:
      filter_items = known_items[user_id]
      final_recs = [item for item in final_recs if item not in filter_items]
    return final_recs[:N]
  return _recs_mapper

Функция `predict_lightfm()` составляет рекомендации для пользователей из test периода на основе обученной LightFM-модели и проставляет соответствующие значения релевантности.

In [ ]:
def predict_lightfm(model, train, test, all_cols, lightfm_mapping, top_N=20, relevances_needed=True):

  # Создаём датафрейм, в котором будут храниться рекомендации для пользователей.
  predictions = pd.DataFrame({"user_id": test["user_id"].unique()})
  predictions = predictions[predictions["user_id"].isin(train["user_id"].unique())]
  known_items = train.groupby("user_id")["item_id"].apply(list).to_dict()

  # Собираем предсказания
  mapper = generate_lightfm_recs_mapper(
    model,
    item_ids=all_cols,
    known_items=known_items,
    N=top_N,
    user_features=None,
    item_features=None,
    user_mapping=lightfm_mapping["users_mapping"],
    item_inv_mapping=lightfm_mapping["items_inv_mapping"],
    num_threads=20
  )

  predictions["item_id"] = predictions["user_id"].map(mapper)
  predictions = predictions.explode("item_id").reset_index(drop=True)
  predictions["rank"] = predictions.groupby("user_id").cumcount() + 1

  if relevances_needed == True:
    test = test.drop(columns="amount")
    test["relevance"] = 1
    predictions = pd.merge(predictions, test, on=["user_id", "item_id"], how="left")
    predictions["relevance"] = predictions["relevance"].fillna(0)
    predictions["relevance"] = predictions["relevance"].astype(int)

    return predictions

  elif relevances_needed == False:
    return predictions

## Обучение бустинга

Подготовим данные для обучения 2 моделей отбора кандидатов. Для оценки качества можно пойти двумя путями.

Если использовать функцию `train_test()`, то в test будут помещены все взаимодействия за 3-ий месяц.

In [ ]:
lfm_train, lfm_pred, test = train_test(interactions, for_boosting=True)

Если же вызвать функцию `train_test_leave_one_out()`, то разбиение на train будет по прицнипу Leave-One-Out и в test окажутся последний взаимодействия выборки пользователей.

In [ ]:
lfm_train, lfm_pred, test = train_test_leave_one_out(interactions)

Преобразуем данные к формату `LightFM.Dataset`.

In [ ]:
interaction_matrix_train, lightfm_mapping, all_cols = make_dataset(lfm_train, lfm_pred)

Обучим LightFM-модели. Параметры заранее подобраны кросс-валидацией. Число скрытых факторов `no_components` выбрано в том числе и с точки зрения скорости/cложности вычислений.

In [ ]:
model_warp_kos = LightFM(
    no_components=100,
    k=3,
    n=11,
    learning_schedule="adagrad",
    loss="warp-kos",
    learning_rate=0.027,
    item_alpha=0.00001,
    user_alpha=0.00014,
    max_sampled=90)

model_warp_kos.fit(interaction_matrix_train, epochs=20)

In [ ]:
model_warp = LightFM(
    no_components=100,
    learning_schedule="adagrad",
    loss="warp",
    learning_rate=0.04,
    item_alpha=0.0001,
    user_alpha=0.00005,
    max_sampled=90
)

model_warp.fit(interaction_matrix_train, epochs=20)

Модели матричной факторизации склонны к переобучению под популярные товары, поэтому для сохранения разнообразия смешения айтемов, полученные в LightFM-модели, необходимо занулить.

In [ ]:
model_warp_kos.item_biases = np.zeros_like(model_warp_kos.item_biases)
model_warp.item_biases = np.zeros_like(model_warp.item_biases)

Функция `candidates_relevances()` объединяет кандидатов от LightFM-моделей первого уровня и проставляет значения релевантности.

In [ ]:
def candidates_relevances(model_first, model_second, lfm_train, lfm_pred, test, all_cols, lightfm_mapping, top_N=30):
  # Отбираем кандидатов
  predictions_first = predict_lightfm(
      model_warp_kos,
      lfm_train,
      lfm_pred,
      all_cols,
      lightfm_mapping,
      top_N=top_N,
      relevances_needed=False
  )

  predictions_second = predict_lightfm(
      model_warp,
      lfm_train,
      lfm_pred,
      all_cols,
      lightfm_mapping,
      top_N=top_N,
      relevances_needed=False
  )

  predictions_first = predictions_first.rename(columns={"rank": "rank_first"})
  predictions_second = predictions_second.rename(columns={"rank": "rank_second"})

  # Объединяем наборы кандидатов.
  predictions_first = pd.merge(
      predictions_first,
      predictions_second,
      on=["user_id", "item_id"],
      how="outer"
  )

  # Проставляем значения релевантности.
  test = test.drop(columns="amount")
  test["relevance"] = 1
  predictions_first = pd.merge(predictions_first, test, on=["user_id", "item_id"], how="left")
  predictions_first["relevance"] = predictions_first["relevance"].fillna(0)
  predictions_first["relevance"] = predictions_first["relevance"].astype(int)

  return predictions_first

Отберём по 30 кандидатов.

In [ ]:
predictions_train = candidates_relevances(
    model_warp_kos,
    model_warp,
    lfm_train,
    lfm_pred,
    test,
    all_cols,
    lightfm_mapping,
    top_N=30
)

<ipython-input-28-63b6dee3084e>:27: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  predictions_first = pd.merge(


Составим датасет для обучения бустинга.

Функция `get_query_id()` нужна для разбиения пользователей по группам для бустинга.

In [ ]:
def get_query_id(df):
  query_map = {}

  for query_id, user_id in enumerate(df["user_id"].unique()):
    query_map[user_id] = query_id

  query_id = df["user_id"].map(query_map)

  return query_id

Функция `positive_negative_sampling()` делает семплирование релевантных и нерелевантных взаимодействий в указанном соотношении и составляет датасеты для обучения и валидации бустинга.

In [ ]:
def positive_negative_sampling(candidates, lfm_pred, pos_neg_ratio=0.2, val_users_size=0.3):
  # Семплируем в требуемом соотношении.
  pos = candidates.merge(
      lfm_pred,
      on=["user_id", "item_id"],
      how="inner"
  )
  pos["target"] = 1

  neg = candidates.set_index(["user_id", "item_id"]) \
          .join(lfm_pred.set_index(["user_id", "item_id"]))
  neg = neg.reset_index()

  neg_sample_frac = len(pos) / (len(neg) * pos_neg_ratio)
  neg = neg.sample(frac=neg_sample_frac)
  neg["target"] = 0

  # Собираем датасеты для обучения и валидации, т.е. для механизма early_stopping.
  train_users, val_users = train_test_split(
      lfm_pred["user_id"].unique(),
      random_state=42,
      test_size=val_users_size
  )

  select_col = ["user_id", "item_id", "rank_first", "rank_second", "target"]
  lgbm_train = shuffle(pd.concat([pos[pos["user_id"].isin(train_users)],
                                  neg[neg["user_id"].isin(train_users)]])[select_col])
  lgbm_val = shuffle(pd.concat([pos[pos["user_id"].isin(val_users)],
                                neg[neg["user_id"].isin(val_users)]])[select_col])

  # Делаем разбиение по пользователям.
  lgbm_train["query_id"] = get_query_id(lgbm_train)
  lgbm_val["query_id"] = get_query_id(lgbm_val)

  train_group = lgbm_train["query_id"].value_counts().sort_index().values
  val_group = lgbm_val["query_id"].value_counts().sort_index().values

  del lgbm_train["query_id"]
  del lgbm_val["query_id"]
  gc.collect()

  # Убедимся, что с типом данных всё в порядке.
  lgbm_train["item_id"] = lgbm_train["item_id"].astype(np.int64)
  lgbm_val["item_id"] = lgbm_val["item_id"].astype(np.int64)

  # Преобразуем данные для обучения и механизма ранней остановки к нужному для lgbm формату.
  train_lgbm_dataset = lgbm.Dataset(
      data=lgbm_train.drop(columns="target"), label=lgbm_train["target"],
      group=train_group
  )

  val_lgbm_dataset = lgbm.Dataset(
      data=lgbm_val.drop(columns="target"), label=lgbm_val["target"],
      group=val_group
  )

  return train_lgbm_dataset, val_lgbm_dataset

In [ ]:
train_lgbm_dataset, val_lgbm_dataset = positive_negative_sampling(
    predictions_train,
    lfm_pred,
    pos_neg_ratio=0.2,
    val_users_size=0.2
)

In [ ]:
del predictions_train

gc.collect()

0

Обучим бустинг.

In [ ]:
params = {
    "objective": "lambdarank",
    "learning_rate": 0.09,
    "n_estimators": 1000,
    "max_depth": 35,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "first_metric_only": True,
    "metric": (
        "lambdarank", "map", "auc"
    ),
    "reg_lambda": 0.0011,
    "eval_at": (20)
}

In [ ]:
booster = lgbm.train(
    params=params,
    train_set=train_lgbm_dataset,
    num_boost_round=1600,
    valid_sets=[train_lgbm_dataset, val_lgbm_dataset],
    early_stopping_rounds=300,
    verbose_eval=50
)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Total groups: 411226, total data: 4111302
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.178311 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 594
[LightGBM] [Info] Number of data points in the train set: 4111302, number of used features: 4
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Total groups: 102805, total data: 1027896
Training until validation scores don't improve for 300 rounds
[50]	training's ndcg@20: 0.772635	training's map@20: 0.668162	training's auc: 0.730176	valid_1's ndcg@20: 0.772809	valid_1's map@20: 0.6

## Оценка гибридной модели

Отберём по 30 кандидатов для каждого пользователя для предсказания на test периоде.

In [ ]:
predictions_first_test = predict_lightfm(
    model_warp_kos,
    lfm_train,
    test,
    all_cols,
    lightfm_mapping,
    top_N=30,
    relevances_needed=False
)

In [ ]:
del model_warp_kos

gc.collect()

0

In [ ]:
predictions_second_test = predict_lightfm(
    model_warp,
    lfm_train,
    test,
    all_cols,
    lightfm_mapping,
    top_N=30,
    relevances_needed=False
)

In [ ]:
del model_warp
del lfm_train, all_cols, lightfm_mapping

gc.collect()

0

In [ ]:
predictions_first_test = predictions_first_test.rename(columns={"rank": "rank_first"})
predictions_second_test = predictions_second_test.rename(columns={"rank": "rank_second"})

Объединим кандидатов.

In [ ]:
predictions_test = pd.merge(
    predictions_first_test,
    predictions_second_test,
    on=["user_id", "item_id"],
    how="outer"
)

<ipython-input-41-c0ccc2a80e65>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  predictions_test = pd.merge(


In [ ]:
predictions_test["item_id"] = predictions_test["item_id"].astype(np.int64)

In [ ]:
del predictions_first_test, predictions_second_test, lfm_pred
del train_test, transform_interactions, make_dataset, generate_lightfm_recs_mapper
del predict_lightfm, candidates_relevances, get_query_id, positive_negative_sampling
del interaction_matrix_train, interactions, params, path, train_lgbm_dataset, val_lgbm_dataset

gc.collect()

0

Посчитаем прогнозы скоров от бустинга.

In [ ]:
predictions_test["lgbm_pred"] = booster.predict(predictions_test)

In [ ]:
del booster

gc.collect()

4

Проставим значения релевантности.

In [ ]:
test = test.drop(columns="amount")
test["relevance"] = 1

In [ ]:
predictions_test = predictions_test.sort_values(
    by=["user_id", "lgbm_pred"], ascending=[True, False])

In [ ]:
predictions_test = pd.merge(
    predictions_test,
    test,
    on=["user_id", "item_id"],
    how="left"
)

In [ ]:
predictions_test["relevance"] = predictions_test["relevance"].fillna(0)
predictions_test["relevance"] = predictions_test["relevance"].astype(int)

In [ ]:
predictions_test.head()

,user_id,item_id,rank_first,rank_second,lgbm_pred,relevance
0,3,69,15.0,7.0,1.409048,0
1,3,11,5.0,6.0,1.313342,0
2,3,192,NaN,15.0,1.159265,0
3,3,41,2.0,10.0,1.058938,0
4,3,165,1.0,11.0,1.035075,0


In [ ]:
predictions_test.user_id.nunique(), predictions_test.item_id.nunique()

(514071, 1728)

Достанем необходимые векторы значений.

In [ ]:
preds_array, relevances_array, recommended_items_ids_array, relevant_items_ids_array = extract_vectors(
    predictions_test,
    test,
    rank_column_name="lgbm_pred"
)

100%|██████████| 514071/514071 [00:18<00:00, 28294.02it/s]
<ipython-input-5-7caced98b2e2>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ranks_array = np.array(ranks_array)
<ipython-input-5-7caced98b2e2>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  relevances_array = np.array(relevances_array)
<ipython-input-5-7caced98b2e2>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must sp

Наконец, посмотрим на качество бустинга.

На отложенном месяце:

In [ ]:
get_metrics(
    preds_array,
    relevances_array,
    recommended_items_ids_array,
    relevant_items_ids_array,
    is_higher_better=True,
    k=20
)

100%|██████████| 514071/514071 [00:02<00:00, 240339.42it/s]


{'MAP@20': 0.2500176385052239,
 'MAR@20': 0.10536632559289105,
 'NDCG@20': 0.31910651845312094,
 'HitRate@20': 0.08224467048326073}

На основании LeaveOneOut-стратегии:

In [ ]:
get_metrics(
    preds_array,
    relevances_array,
    recommended_items_ids_array,
    relevant_items_ids_array,
    is_higher_better=True
)

100%|██████████| 456890/456890 [00:01<00:00, 428857.97it/s]


{'MAP@20': 0.06629452961350915,
 'MAR@20': 0.22583554028321914,
 'NDCG@20': 0.10116652168922431,
 'HitRate@20': 0.011291777014182852}